In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_2179/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/21 11:52:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/21 11:52:27 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
import pandas as pd

# Sample data for Inventory table
inventory_data = {
    'item_id': [1374, 4245, 5743, 8543, 2556, 2452, 3255, 1672, 4256, 6325],
    'item_type': ['prime_eligible', 'not_prime', 'prime_eligible', 'not_prime', 'not_prime', 'prime_eligible', 'not_prime', 'prime_eligible', 'prime_eligible', 'prime_eligible'],
    'item_category': ['Watches', 'Art', 'Software', 'Clothing', 'Shoes', 'Scientific', 'Furniture', 'Beauty', 'Furniture', 'Food'],
    'square_footage': [68.00, 26.40, 325.00, 64.50, 15.00, 85.00, 22.60, 8.50, 55.50, 13.20]
}

# Create DataFrame for Inventory
inventory_df = pd.DataFrame(inventory_data)

# Display the DataFrame
print("Inventory table:")
print(inventory_df)



df_person = spark.createDataFrame(inventory_df)
df_person.createOrReplaceTempView("Inventory")



Inventory table:
   item_id       item_type item_category  square_footage
0     1374  prime_eligible       Watches            68.0
1     4245       not_prime           Art            26.4
2     5743  prime_eligible      Software           325.0
3     8543       not_prime      Clothing            64.5
4     2556       not_prime         Shoes            15.0
5     2452  prime_eligible    Scientific            85.0
6     3255       not_prime     Furniture            22.6
7     1672  prime_eligible        Beauty             8.5
8     4256  prime_eligible     Furniture            55.5
9     6325  prime_eligible          Food            13.2


In [41]:
query = """
    with t as (
        select
            item_type,
            sum(square_footage) as total_square,
            count(*) as total_item
        from Inventory i 
        group by item_type
    ),
    
    remain_square as (
        select
            *,
            500000 - max_item * total_square as remain_square
        from (
            select
                item_type,
                total_square,
                total_item,
                cast(500000 / total_square as int) as max_item
            from t
        )
    )
    
    select 
        'not_prime' as item_type, 
        cast(
            (select remain_square from remain_square where item_type = 'prime_eligible' limit 1) 
            / 
            (select total_square from t where item_type = 'not_prime' limit 1) as int    
        ) * (select total_item from t where item_type = 'not_prime' limit 1) as item_count
    union all
    select
        'prime_eligible' as item_type,
        cast(
            500000 
            / 
            (select total_square from t where item_type = 'prime_eligible' limit 1) as int
        ) * (select total_item from t where item_type = 'prime_eligible' limit 1) as item_count
    
    
"""

In [42]:
spark.sql(query).show()

+--------------+----------+
|     item_type|item_count|
+--------------+----------+
|     not_prime|         8|
|prime_eligible|      5400|
+--------------+----------+

